# Building a Metaphor Data Agent

This tutorial walks through using the LLM tools provided by the [Metaphor API](https://platform.metaphor.systems/) to allow LLMs to easily search and retrieve HTML content from the Internet.

To get started, you will need an [OpenAI api key](https://platform.openai.com/account/api-keys) and a [Metaphor API key](https://dashboard.metaphor.systems/overview)

We will import the relevant agents and tools and pass them our keys here:

In [ ]:
# Set up OpenAI
import os

os.environ["OPENAI_API_KEY"] = "sk-your-key"

from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI

# Set up Metaphor tool
from llama_index.tools.metaphor.base import MetaphorToolSpec

metaphor_tool = MetaphorToolSpec(
    api_key="your-key",
)

metaphor_tool_list = metaphor_tool.to_tool_list()
for tool in metaphor_tool_list:
    print(tool.metadata.name)

search
retrieve_documents
search_and_retrieve_documents
find_similar
current_date


## Testing the Metaphor tools

We've imported our OpenAI agent, set up the api key, and initialized our tool, checking the methods that it has available. Let's test out the tool before setting up our Agent.

All of the Metaphor search tools make use of the `AutoPrompt` option where Metaphor will pass the query through an LLM to refine and improve it.

In [ ]:
metaphor_tool.search("machine learning transformers", num_results=3)

[Metaphor Tool] Autoprompt: Here's a great resource for learning about machine learning transformers:


[{'title': 'Natural Language Processing with Transformers Book',
  'url': 'https://transformersbook.com/',
  'id': 'm0fnocgD1zPLxtDZuGt5JQ'},
 {'title': 'Transformers',
  'url': 'https://www.nlpdemystified.org/course/transformers',
  'id': 'jPHVA37dax24EwEan9jj0g'},
 {'title': 'transformersbook (Natural Language Processing with Transformers)',
  'url': 'https://huggingface.co/transformersbook',
  'id': 'lcsZRBBHevoB4wAn1SFMtA'}]

In [ ]:
metaphor_tool.retrieve_documents(["iEYMai5rS9k0hN5_BH0VZg"])

[Document(id_='4ab1448d-663a-4dfd-8dc0-dd7ba860e42d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='bbd2ad7f78cf02100add6077decdf885ffebfe37743bfc13e80e61b5b833e46c', text='<div><div><p>\n <a href="/static/transformers-bumblebee-d4ec62a6a46aae8f83b7419242f59f4e-48854.jpg">\n \n \n \n \n \n \n \n </a>\n </p>\n<p>In this post, we will be describing a class of sequence processing models known as Transformers (…<em>robots in disguise</em>).\nJokes aside, Transformers came out on the scene not too long ago and have rocked the natural language processing community because of\ntheir pitch: state-of-the-art and efficient sequence processing <strong>without recurrent units or convolution</strong>. </p>\n<p><em>“No recurrent units or convolution?! What are these models even made of?!”</em>, you may be exclaiming to unsuspecting strangers on the streets. </p>\n<p>Not much it turns out, other than a bunch of attention and feedf

In [ ]:
metaphor_tool.find_similar(
    "https://www.mihaileric.com/posts/transformers-attention-in-disguise/"
)

[{'title': 'Transformers: Attention in Disguise',
  'url': 'https://www.mihaileric.com/posts/transformers-attention-in-disguise/',
  'id': 'iEYMai5rS9k0hN5_BH0VZg'},
 {'title': 'Transformers: a Primer',
  'url': 'http://www.columbia.edu/~jsl2239/transformers.html',
  'id': 'cXbT9IsR5u8NtLTAIcUWOA'},
 {'title': 'Illustrated Guide to Transformers- Step by Step Explanation',
  'url': 'https://towardsdatascience.com/illustrated-guide-to-transformers-step-by-step-explanation-f74876522bc0?gi=8fe76db5c4d9',
  'id': 'czTCPZ1vqo-f92WQwKvRig'}]

In [ ]:
metaphor_tool.search_and_retrieve_documents(
    "This is the best explanation for machine learning transformers:", num_results=1
)

[Document(id_='d6408474-33d9-451b-9742-0634b8aea667', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='37e68a115716214b23b68f40a70cca20943f3161fc6b9a1bed5ab21347d0a41e', text='Dale’s Blog → https://goo.gle/3xOeWoK\nClassify text with BERT → https://goo.gle/3AUB431\n\nOver the past five years, Transformers, a neural network architecture, have completely transformed state-of-the-art natural language processing. Want to translate text with machine learning? Curious how an ML model could write a poem or an op ed? Transformers can do it all. In this episode of Making with ML, Dale Markowitz explains what transformers are, how they work, and why they’re so impactful. Watch to learn how you can start using transformers in your app!\n\nChapters: \n0:00 - Intro\n0:51 - What are transformers?\n3:18 - How do transformers work?\n7:41 - How are transformers used?\n8:35 - Getting started with transformers\n\nWatch more episodes of M

We can see we have different tools to search for results, retrieve the results, find similar results to a web page, and finally a tool that combines search and document retrieval into a single tool. We will test them out in LLM Agents below:

### Using the Search and Retrieve documents tools in an Agent

We can create an agent with access to the above tools and start testing it out:

In [ ]:
# We don't give the Agent our unwrapped retrieve document tools, instead passing the wrapped tools
agent = FunctionAgent(
    tools=metaphor_tool_list,
    llm=OpenAI(model="gpt-4.1"),
)

# Context to store chat history
ctx = Context(agent)

In [ ]:
print(await agent.run("What are the best resturants in toronto?", ctx=ctx))

In [ ]:
print(await agent.run("tell me more about Osteria Giulia", ctx=ctx))

## Avoiding Context Window Issues

The above example shows the core uses of the Metaphor tool. We can easily retrieve a clean list of links related to a query, and then we can fetch the content of the article as a cleaned up html extract. Alternatively, the search_and_retrieve_documents tool directly returns the documents from our search result.

We can see that the content of the articles is somewhat long compared to current LLM context windows, and so to allow retrieval and summary of many documents we will set up and use another tool from LlamaIndex that allows us to load text into a VectorStore, and query it for retrieval. This is where the `search_and_retrieve_documents` tool become particularly useful. The Agent can make a single query to retrieve a large number of documents, using a very small number of tokens, and then make queries to retrieve specific information from the documents.

In [ ]:
from llama_index.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec

# The search_and_retrieve_documents tool is the third in the tool list, as seen above
wrapped_retrieve = LoadAndSearchToolSpec.from_defaults(
    metaphor_tool_list[2],
)

Our wrapped retrieval tools separate loading and reading into separate interfaces. We use `load` to load the documents into the vector store, and `read` to query the vector store. Let's try it out again

In [ ]:
wrapped_retrieve.load("This is the best explanation for machine learning transformers:")
print(wrapped_retrieve.read("what is a transformer"))
print(wrapped_retrieve.read("who wrote the first paper on transformers"))

[Metaphor Tool] Autoprompt: Here is a great article explaining machine learning transformers:

A transformer is a type of artificial neural network used in natural language processing (NLP) tasks such as machine translation, text summarization, and question answering. It is based on the Transformer architecture, which was introduced in 2017 by Google researchers. The Transformer architecture uses attention mechanisms to learn the relationships between words in a sentence, allowing it to better understand the context of the sentence and generate more accurate translations.

The first paper on transformers was written by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, and Illia Polosukhin in 2017. The paper was titled "Attention Is All You Need".


## Creating the Agent

We now are ready to create an Agent that can use Metaphors services to it's full potential. We will use our wrapped read and load tools, as well as the `get_date` utility for the following agent and test it out below:

In [ ]:
# Just pass the wrapped tools and the get_date utility
agent = FunctionAgent(
    tools=[*wrapped_retrieve.to_tool_list(), metaphor_tool_list[4]],
    llm=OpenAI(model="gpt-4.1"),
)

In [ ]:
print(
    await agent.run(
        "Can you summarize everything published in the last month regarding news on"
        " superconductors"
    )
)

We asked the agent to retrieve documents related to superconductors from this month. It used the `get_date` tool to determine the current month, and then applied the filters in Metaphor based on publication date when calling `search`. It then loaded the documents using `retrieve_documents` and read them using `read_retrieve_documents`.

We can make another query to the vector store to read from it again, now that the articles are loaded: